In [1]:
import nltk
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import pandas as pd

In [ ]:
from ipynb.fs.full.preprocessing import * 
from ipynb.fs.full.WordProcessing import * 

In [ ]:
data = pd.read_csv(f_in)

In [ ]:
analyzer = SentimentAnalyzer()

In [ ]:
analyzer.save_file(analyzer, 'sentiment_analyzer')